## 14.1 测试 stdout 输出

## 14.9 捕获异常后抛出另外的异常

你想捕获一个异常后抛出另外一个不同的异常,同时还得在异常回溯中保留两个
异常的信息。

为了链接异常,使用 `raise from` 语句来代替简单的 `raise` 语句。它会让你同时保
留两个异常的信息。

In [ ]:
>>> def example():
 try:
     int('N/A')
 except ValueError as e:
     raise RuntimeError('A parsing error occurred') from e
>>> example()
Traceback (most recent call last):
File "<stdin>", line 3, in example
ValueError: invalid literal for int() with base 10: 'N/A'

上面的异常是下面的异常产生的直接原因:

In [ ]:
Traceback (most recent call last):
File "<stdin>", line 1, in <module>
File "<stdin>", line 5, in example
RuntimeError: A parsing error occurred
>>>

还可以通过查看异常对象的 __cause__ 属性来跟踪异常
链

In [ ]:
try:
    func()
except RuntimeError as e:
    print("it donot work: ", e)
    if e.__cause__:
        print("cause is ", e.__cause__)

it donot work:  failed to parse int
cause is  invalid literal for int() with base 10: 'N/A'

在设计代码时,在另外一个 except 代码块中使用 raise 语句的时候你要特别小心
了。大多数情况下,这种 raise 语句都应该被改成 raise from 语句。也就是说你应该
使用下面这种形式:

In [ ]:
try:
    ...
except SomeException as e:
    raise DifferentException() from e

这样做的原因
**是你应该显示的将原因链接起来。也就是说,DifferentException是直接从 SomeException 衍生而来。这种关系可以从回溯结果中看出来。**

## 14.10 重新抛出被捕获的异常

想将异常传播下去。一个很常见的用法是在捕获所有异常的处理器中:

In [ ]:
try:
    ...
except Exception as e:
    # Process exception information in some way
    ...
    # Propagate the exception
    raise

## 14.11 输出警告信息

要输出一个警告消息,可使用 `warning.warn()` 函数。

warn() 的**参数是一个警告消息和一个警告类**,警告类有如下几种:`UserWarning,DeprecationWarning, SyntaxWarning, RuntimeWarning, ResourceWarning, 或 Future-Warning.`

对警告的处理取决于你如何运行解释器以及一些其他配置。例如,如果你使用 `-Wall `选项去运行 Python,你会得到如下的输出:

In [ ]:
bash % python3 -W all example.py
example.py:5: DeprecationWarning: logfile argument is deprecated
warnings.warn('logfile argument is deprecated', DeprecationWarning)

通常来讲,警告会输出到标准错误上。如果你想讲警告转换为异常,可以使用`error` 选项:

In [ ]:
bash % python3 -W error example.py
Traceback (most recent call last):
File "example.py", line 10, in <module>
func(2, 3, logfile='log.txt')
File "example.py", line 5, in func
warnings.warn('logfile argument is deprecated', DeprecationWarning)
DeprecationWarning: logfile argument is deprecated
bash %

在你维护软件,提示用户某些信息,但是又不需要将其上升为异常级别,那么输出
警告信息就会很有用了。

## 14.12 调试基本的程序崩溃错误

如果你的程序因为某个异常而崩溃,运行 `python3 -i someprogram.py` **可执行简
单的调试**。`-i 选项可让程序结束后打开一个交互式 shell`。

当程序崩溃后，再次使用`python -i`运行python程序，在程序崩溃后打开 Python 的调试器，使用` import pdb`调试代码，打印堆栈，变量等。

In [ ]:
>>> import pdb
>>> pdb.pm()
> sample.py(4)func()
-> return n + 10
(Pdb) w
sample.py(6)<module>()
-> func('Hello')
> sample.py(4)func()
-> return n + 10
(Pdb) print n
'Hello'
(Pdb) q
>>>

如果你的代码所在的环境很难获取交互shell(比如在某个服务器上面), 通常可以
捕获异常后自己打印跟踪信息。

In [ ]:
import sys
import traceback

try:
    func(arg)
except:
    print('**** AN ERROR OCCURRED ****')
    traceback.print_exc(file=sys.stderr)

要是你的程序没有崩溃,而只是产生了一些你看不懂的结果,你在感兴趣的地方插
入一下 `print() 语句也是个不错的选择`。不过,要是你打算这样做,有一些小技巧可
以帮助你。首先,`traceback.print_stack()` 函数会**在程序运行到那个点的时候创建
一个跟踪栈**。例如:

还可以像下面这样使用 `pdb.set_trace()` 在任何地方手动的启动调试器

In [ ]:
import pdb
def func(arg):
    ...
    pdb.set_trace()
    ...

## 14.13 给你的程序做性能测试

## 14.14 加速程序运行

关于程序优化的第一个准则是“不要优化”,

第二个准则是“不要优化那些无关紧
要的部分”。

如果你的程序运行缓慢,首先你得使用`给你的程序做性能测试`技术先对它进行性能
测试找到问题所在。

`通常来讲你会发现你得程序在少数几个热点地方花费了大量时间`,比如内存的数
据处理循环。一旦你定位到这些点,你就可以使用下面这些实用技术来加速程序运行

### 定义在全局范围的代码运行起来要比定义在函数中运行慢的多,20~30%提升
这种速度差异是由于局部变量和全局变量的实现方式(使用局部变量要更快些)。
因此,如果你想让程序运行更快些,只需要将脚本语句放入函数中即可:

In [ ]:
# 未知优化， 使用了全局变量，比较慢
# somescript.py
import csv
import sys

with open(sys.argv[1]) as f:
    for row in csv.reader(f):
        # Some kind of processing
        pass


In [ ]:
# 优化后的程序
import sys
import csv

def main(filename):
    with open(filename) as f:
        for row in csv.reader(f):
            # Some kind of processing
            pass

main(sys.argv[1])

### 尽可能去掉属性访问, 20~30%提升，仅在多次循环中有效

每一次使用`点 (.) 操作符`来访问属性的时候会`带来额外的开销`。它会触发特定的方法,比如 __getattribute__() 和 __getattr__() ,这些方法会进行字典操作操作。


通常你`可以使用 from module import name` 这样的导入形式,以及使用绑定的方法。

假设你有如下的代码片段:

In [ ]:
import math

def compute_roots(nums):
    result = []
    for n in nums:
        result.append(math.sqrt(n))
    return result

# Test
nums = range(1000000)
for n in range(100):
    r = compute_roots(nums)

In [ ]:
from math import sqrt

def compute_roots(nums):
result = []
result_append = result.append
for n in nums:
result_append(sqrt(n))
return result

。在内部循环中,可以将某个需要频繁访问的属
性放入到一个局部变量中

> **最好的性能优化是从不工作到
工作状态的迁移”。直到你真的需要优化的时候再去考虑它。确保你程序正确的运行通
常比让它运行更快要更重要一些**